In [1]:
from model import REModel
from transformers import ElectraTokenizer,ElectraConfig,ElectraModel
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification, Trainer, TrainingArguments, RobertaConfig, RobertaTokenizer, RobertaForSequenceClassification, BertTokenizer,EarlyStoppingCallback, AutoModel

In [2]:
s = REModel(pretrained_id = 'klue/roberta-large')

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
tmp = AutoConfig.from_pretrained('klue/bert-base')
tmp

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.10.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

In [10]:
tmp = AutoConfig.from_pretrained("monologg/koelectra-base-v3-discriminator")
tmp

ElectraConfig {
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "transformers_version": "4.10.0",
  "type_vocab_size": 2,
  "vocab_size": 35000
}

In [11]:
tmp = AutoModelForSequenceClassification.from_pretrained('klue/roberta-large')
tmp

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifi

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
         

In [1]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-large')

In [14]:
sub = '사람과 기관'
obj = '기관 그리고 기관'
text = sub + ' ' + obj
len(tokenizer(' ',padding='max_length',max_length = 32)['input_ids'])

32

In [37]:
import numpy as np
sub = tokenizer.encode('@*사람*김한성@',add_special_tokens=False)
obj = tokenizer.encode('#^장소^학교#',add_special_tokens=False)

In [38]:
sent = tokenizer('@*사람*김한성@ 은 나의 사랑이자 #^장소^학교#입니다.')['input_ids']

In [45]:
def get_embed_idx(sent,ent1,ent2):
    embed = [0 for _ in range(len(sent))]
    for i,b in enumerate(sent):
        try:
            if b == ent1[0] and sent[i+len(ent1)-1]== ent1[-1]:
                s_s,s_e = i,i+len(ent1)-1
            if b == ent2[0] and sent[i+len(ent2)-1]== ent2[-1]:
                o_s,o_e = i,i+len(ent2)-1
        except:
            continue

    embed[s_s:s_e+1] = [1 for _ in range(end-start+1)]
    embed[o_s:o_e+1] = [1 for _ in range(end-start+1)]
    return embed,[s_s,s_e,o_s,o_e]

In [46]:
get_embed_idx(sent,sub,obj)

([0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
 [1, 7, 14, 19])

In [36]:
sent

[0, 36, 14, 3611, 14, 12476, 2047, 36, 2]

In [48]:
a =[0 for _ in range(10)]
a[2:5] = [1 for _ in range(5-2)]
a

[0, 0, 1, 1, 1, 0, 0, 0, 0, 0]

In [52]:
import torch
[1]==torch.tensor([1])

False

In [38]:
def get_embed_idx(sent):
    sep = tokenizer.convert_tokens_to_ids(':')
    if sent[0]==0 and sent[1]==2:
        return [0,1,0,1]
    i=0
    s_s,s_e,o_s,o_e = 1,0,0,0
    while sent[i] != 2:
        i+=1
        if sent[i] == sep:
            s_e = i-1
            o_s = i+1
    o_e = i-1
    return [s_s,s_e,o_s,o_e]


In [41]:
sent = tokenizer(' ')['input_ids']

In [42]:
get_embed_idx(sent)

[0, 1, 0, 1]

In [43]:
sent

[0, 2]

In [47]:
a = [ i for i in range(10)]
b = [ 1 for i in range(10)]
c = [2 for i in range(10)]
d = [10 for i in range(10)]
def la(a,b,c,d):
    return a+b+c+d
list(map(la,a,b,c,d))
    

[13, 14, 15, 16, 17, 18, 19, 20, 21, 22]

In [71]:
import numpy as np
a = np.array([[ i for i in range(10)],[ i+1 for i in range(10)]])
a.shape

(2, 10)

In [63]:
b = []
b.append(a)

In [64]:
b

[array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
        [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]])]

In [68]:
c = np.array(b)

In [70]:
c.argmax(2)

array([[9, 9]])

In [72]:
d = [2 for i in range(10)]
np.array(d).shape

(10,)